## COM361 &mdash; Introdução à Otimização &mdash; 2023, Prof. Amit ##

# Descobrindo equações governantes a partir de dados por identificação esparsa de sistemas dinâmicos não lineares. #

#### Leonardo Soares da Costa Tanaka (leonardo.tanaka@poli.ufrj.br), Lincoln Rodrigues Proença (lincoln22220@poli.ufrj.br )

*****

### Índice

1. [Introdução](#1.-Introdução)
1. [Modelo Matemático - Sparse Identification of Nonlinear Dynamics (SINDy)](#1.-Modelo_matemático)
1. [Solução](#3.-Solução)
1. [Resultados e Discussão](#4.-Resultados-e-discussão)
  1. [Subseção Opcional](#4.A.-Acrescente-subseções-se-necessário)
1. [Conclusão](#5.-Conclusão)
1. [Referências bibliográficas](#6.-Referências_bibliográficas)

## 1. Introdução ##

As primeiras frases devem dar um panorama geral do projeto. Em seguida, uma descrição mais detalhada do problema que será resolvido, uma breve história do surgimento do problema (com [citations](https://en.wikipedia.org/wiki/Citation)) que será abordado, porque é importante e interessante e quaisquer outros fatos correlatos que queira expor. A fonte dos dados utilizados deve ser explicitada (pesquisa? internet? gerados sinteticamente?). Também deve ser feito um esboço do restante do relatório.


Esta seção deve ter entre 300 e 600 palavras, e **deve ser acessível a leigos** (não deve ser suposto que o leitor desta seção cursou esta disciplina). Podem incluir figuras se acharem convenientes:

![fixit flowchart][flow]

Para mais ajuda no uso de Markdown, veja [este link](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

[flow]: https://s-media-cache-ak0.pinimg.com/736x/f5/75/c5/f575c53b93724808c6f0211890a54900.jpg

## 2. Modelo matemático - Sparse Identification of Nonlinear Dynamics (SINDy) ##

Esta seção deve conter uma discussão das hipóteses de modelagem feitas no problema (conforme a origem do problema: física? economia? redes sociais? ...). Explique a escolha das variáveis de decisão, as restrições e a função objetivo. Finalmente, mostre o problema de otimização escrite em forma padrão. Discute o tipo de modelo adotado (LP, QP, MIP, etc.). Equações devem ser formatadas em $\LaTeX$ dentro do notebook Julia. Nesta seção, pode supor que **o leitor está familiarizado com a matéria"**.

Aqui, consideramos sistemas dinâmicos na forma:

$$
\begin{equation*}
\dot{\mathbf{x}} = \frac{d}{dt}\mathbf{x}(t) = \mathbf{f}(\mathbf{x}(t))
\end{equation*}
$$

O vetor $x(t) \in \mathbb{R}^n$ denota o estado de um sistema no tempo $t$, e a função $f(x(t))$ representa as restrições dinâmicas que definem as equações de movimento do sistema, como a segunda lei de Newton.

Para determinar a função $f$ a partir de dados, coletamos um histórico temporal do estado $x(t)$ e medimos a derivada $\dot{x}(t)$ ou a aproximamos numericamente a partir de $x(t)$. Os dados são amostrados em vários momentos $t_1, t_2, \ldots, t_m$ e organizados em duas matrizes:

$$
\begin{equation*}
\mathbf{X} = \begin{bmatrix}
\mathbf{x}^T(t_1) \\ \mathbf{x}^T(t_2) \\ \vdots \\ \mathbf{x}^T(t_m)
\end{bmatrix} =
\begin{bmatrix}
x_1(t_1) & x_2(t_1) & \cdots & x_n(t_1) \\
x_1(t_2) & x_2(t_2) & \cdots & x_n(t_2) \\
\vdots   & \vdots   & \ddots & \vdots \\
x_1(t_m) & x_2(t_m) & \cdots & x_n(t_m)
\end{bmatrix}
\end{equation*}
$$

$$
\begin{equation*}
\dot{\mathbf{X}} = \begin{bmatrix}
\dot{\mathbf{x}}^T(t_1) \\ \dot{\mathbf{x}}^T(t_2) \\ \vdots \\ \dot{\mathbf{x}}^T(t_m)
\end{bmatrix} =
\begin{bmatrix}
\dot{x}_1(t_1) & \dot{x}_2(t_1) & \cdots & \dot{x}_n(t_1) \\
\dot{x}_1(t_2) & \dot{x}_2(t_2) & \cdots & \dot{x}_n(t_2) \\
\vdots   & \vdots   & \ddots & \vdots \\
\dot{x}_1(t_m) & \dot{x}_2(t_m) & \cdots & \dot{x}_n(t_m)
\end{bmatrix}
\end{equation*}
$$

Em seguida, construímos uma biblioteca $\Theta(\mathbf{X})$ composta por funções não lineares candidatas das colunas de $\mathbf{X}$. Por exemplo, $\Theta(\mathbf{X})$ pode consistir em termos constantes, polinomiais e trigonométricos:

$$
\begin{equation}
\bf{\Theta}(\bf{X}) = \begin{bmatrix}
\vdots & \vdots & \vdots & \vdots & & \vdots & \vdots & \\
1 & \bf{X} & \bf{X}^2 & \bf{X}^3 & \cdots & \sin(\bf{X}) & \cos(\bf{X}) & \cdots\\
\vdots & \vdots & \vdots & \vdots & & \vdots & \vdots &
\end{bmatrix}
\end{equation}
$$

Para os exemplos usados, os dados são coletados para o sistema e empilhados em duas grandes matrizes de dados $\mathbf{X}$ e $\dot{\mathbf{X}}$, onde cada linha de $\mathbf{X}$ é um instantâneo do estado $x$ no tempo, e cada linha de $\dot{\mathbf{X}}$
é um instantâneo da derivada temporal do estado $\dot{x}$ no tempo. Aqui, a dinâmica do lado direito é identificada no espaço de polinômios $\Theta(\mathbf{X})$ em $(x, y, z)$ até a quinta ordem, embora outras funções como $\sin$, $\cos$, $\exp$, ou polinômios de ordem superior possam ser incluídas.

$$
\begin{equation}
\Theta(\mathbf{X}) = \begin{bmatrix}
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
x(t) &
y(t) &
z(t) &
x(t)^2 &
x(t)y(t) &
\dots &
z(t)^5 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots
\end{bmatrix}
\end{equation}
$$

O número de possíveis estruturas de modelo a partir desta biblioteca é combinatoriamente alto. $\textbf{f}(\textbf{x}(t))$ é então substituído por $\bf{\Theta}(\bf{X})$ e um vetor de coeficientes $\bf{\Xi}=\left[\bf{\xi}_1 \bf{\xi}_2 \cdots \bf{\xi}_n \right]$ determinando os termos ativos em $\textbf{f}(\textbf{x}(t))$:

$$
\dot{\bf{X}}=\bf{\Theta}(\bf{X})\bf{\Xi}
$$

Porque apenas alguns termos são esperados para serem ativos em cada ponto no tempo, assume-se que $\textbf{f}(\textbf{x}(t))$ admite uma representação esparsa em $\bf{\Theta}(\bf{X})$. Isso se torna então um problema de otimização em encontrar um $\bf{\Xi}$ esparsa que incorpora de forma ótima $\dot{\textbf{X}}$. Em outras palavras, um modelo é obtido realizando uma regressão de mínimos quadrados no sistema  com regularização de promoção de esparsidade ($L_1$).

$$
\bf{\xi}_k=\underset{\bf{\xi}'_k}{\arg\min}\left|\left|\dot{\bf{X}}_k-\bf{\Theta}(\bf{X})\bf{\xi}'_k\right|\right|_2 +\lambda \left|\left|\bf{\xi}'_k\right|\right|_1
$$

onde $\lambda$ é um parâmetro de regularização. Finalmente, o conjunto esparsa de $\bf{\xi}_k$ pode ser usado para reconstruir o sistema dinâmico:

$$
\dot{x}_k=\bf{\Theta}(\bf{x})\bf{\xi}_k
$$



## 3. Solução ##

Nesta seção, coloque seu código em Julia + JuMP e resolva o problema proposto. Seu código deve ser limpo (não macarrônico!), de fácil leitura, bem comentado e anotado e deve compilar sem erros em Julia 1.x, x$\geq 1$! Não valem códigos em outras linguagens. **Vou rodar seu código para avaliar seu projeto**. Sugiro a utilização de múltiplos blocos de códigos separados por blocos de texto (células Markdown) explicando as várias partes da sua solução. Sugiro também a resolução de várias versões do seu problema, com modelos e hipóteses diferentes.

É permitido chamar pacotes externos, mas evite a utilização de bibliotecas exóticas (pois, em geral, não rodam em todas as versões de Julia, e terei que instalar a mesma versão que você usou, ou rodar na plataforma Google Colab, que gostaria de evitar).

In [3]:
# Este é um exemplo de um bloco de código
using JuMP, Clp
m = Model(with_optimizer(Clp.Optimizer,LogLevel=0) )
bichos = [:cavalos, :jegues, :cabras]  # estes são os bichos 
@variable(m, x[bichos] >= 0)          # as quantidades de cada um (não podem ser negativas)
@constraint(m, sum(x) <= 10)          # não podemos ter mais de 10 no total.
@objective(m, Max, x[:cavalos])        # queremos maximizar o número de cavalos
optimize!(m)

for i in bichos
    println("O número total de ", i, " é: ", JuMP.value.(x[i]))     # imprime o resultado na tela
end

O número total de cavalos é: 10.0
O número total de jegues é: 0.0
O número total de cabras é: 0.0


**Tenha certeza de que seu código compila corretamente! Rodarei seu código!**

## 4. Resultados e discussão ##

Neste seção, os resultados obtidos serão exibidos e discutidos. Mostre figuras, gráficos, imagens, curvas de compromisso, e o que mais puder melhor ilustrar seus resultados. A discussão deverá explicar o que significam os resultados e como interpretá-los. As limitações da sua abordagem/modelo também devem ser colocadas, bem como uma análise da sensibilidade dos resultados em relação às hipóteses feitas.


Utilize plots (veja exemplos  `PyPlot` [aqui](https://gist.github.com/gizmaa/7214002))

Aqui está um exemplo de uma tabela (em Markdown):

| Tabelas        | São           | Boas  |
| ------------- |:-------------:| -----:|
| col 3 é      | alinhado à direita |\$1600 |
| col 2 é      | centrado      |  \$12 |
| texto | também serve      |   \$1 |

### 4.A. Subseções devem ser utilizadas para organizar seu texto.

#### 4.A.a. ou até subsubseções.

## 5. Conclusão ##

Faça um resumo do que encontrou e dos seus resultados, e fale de pelo menos uma direção na qual  seu trabalho pode ser desenvolvido no futuro, algo que poderia ser interessante em decorrência do seu projeto.


## 6. Referências bibliográficas ##

[1] Brunton, S. L., Proctor, J. L., & Kutz, J. N. (2016). *Descoberta de equações governantes a partir de dados por identificação esparsa de sistemas dinâmicos não lineares.* *Proceedings of the National Academy of Sciences*, 113(15), 3932-3937. DOI: [10.1073/pnas.1517384113](https://doi.org/10.1073/pnas.1517384113)

[2] Brunton, S. L., Proctor, J. L., & Kutz, J. N. (2016). *Descoberta de equações governantes a partir de dados: Identificação esparsa de sistemas dinâmicos não lineares - Informações de suporte.* *Proceedings of the National Academy of Sciences*, 113(15), 3932-3937. DOI: [10.1073/pnas.1517384113](https://doi.org/10.1073/pnas.1517384113)

[3] Brunton, S. (2023). *Physics Informed Machine Learning Playlist.* [YouTube Playlist](https://www.youtube.com/playlist?list=PLMrJAkhIeNNQ0BaKuBKY43k4xMo6NSbBa). Última atualização em 31 de mar. de 2023.
